In [9]:
import os, boto3
import pandas as pd
import requests, io, sys

if sys.version_info[0] < 3:
    from StringIO import StringIO #Python 2.x
else:
    from io import StringIO #Python 3.x

    
client = boto3.client('s3')

bucket_name = 'sftp-analytics-stg-transferbucket-a3e4afdvn23'


object_key = 'data/output/Unsaved/2019/11/06/234929f6-1f52-4206-bac2-30bf3b2cc198.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
Body = csv_obj['Body']
#csv_string = Body.read().decode('utf-8')


csv_string = Body.read().decode('latin-1')

In [10]:
df = pd.read_csv(StringIO(csv_string),sep=",")
df.head()

,codmes,key_value,monto_ec,tea_ec,codmes2,max_lin_tc_m06,max_lin_tc_m12,min_lin_tc_m01,min_lin_tc_m06,min_lin_tc_m12,...,ctd_camptotm06,max_camptot06m,rec_frstlv06m,grp_frstlv06m,ctd_camptotm04,prob_value,segmento,ing_bruto,ing_neto,orden
0,201907,4593B45C40190267580017493D886741DC1C804B93EB44...,10800.0,31.22,201906,31080.00,31080.00,3800.0,3800.0,2500.0,...,5.0,7.0,5.0,G3,3.0,0.593662,3,4558.0,3647.0,1
1,201909,7B290C54B86126665F8630762305BFF787EEB75CB71FE3...,38100.0,16.63,201908,23781.60,23731.20,13700.0,13700.0,13700.0,...,9.0,9.0,6.0,G3,9.0,0.089537,2,8878.0,6658.0,1
2,201907,C5C79D500A603156826227F156F77CB25A7433C4B37DCB...,6300.0,34.33,201906,1300.00,1300.00,1300.0,1300.0,1300.0,...,6.0,6.0,5.0,G3,6.0,0.327935,2,1380.0,1201.0,1
3,201909,E17495463B39A523078F948CF34F55E96079F5042E6CF8...,5200.0,26.68,201908,15213.57,17507.46,300.0,300.0,300.0,...,5.0,5.0,6.0,G3,3.0,0.420560,3,2138.0,1818.0,1
4,201907,90B681BBDA5B77543213F892D91F90EEA3DD1851032D6D...,4700.0,58.08,201906,26984.00,9400.00,1800.0,1800.0,1801.0,...,5.0,5.0,3.0,G2,4.0,0.281577,3,2051.0,1744.0,1


In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import backend as K
import keras.callbacks as callbacks
from keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.regularizers import L1L2
from keras.layers import BatchNormalization, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from scipy.optimize import minimize
from sklearn.metrics import log_loss
from pandas import ExcelWriter
from keras.models import load_model
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline

Using TensorFlow backend.


In [35]:
df_matriz_2 = df[(df.segmento==2) & (df.monto_ec<=16000)]

df_matriz_2['mes_ultprd'] = df_matriz_2['mes_ultprd'].fillna(df_matriz_2.codmes)
df_matriz_2['rec_ultprd'] = ((pd.to_datetime(df_matriz_2['codmes'], format='%Y%m') - pd.to_datetime(df_matriz_2['mes_ultprd'].astype('int').astype('str'), format='%Y%m')) / np.timedelta64(1, 'D')).astype('int64') 

df_matriz_2['sow_ibk'] = df_matriz_2.salvig_tot_rccibk_m01/df_matriz_2.salvig_tot_rccsf_m01
df_matriz_2['cem_2'] = df_matriz_2.salvig_tot_rccsf_m01/df_matriz_2.ing_bruto

df_matriz_2.rec_frstlv06m = df_matriz_2.rec_frstlv06m.fillna(0)
df_matriz_2['rec_frstlv06m'] = np.where(df_matriz_2['rec_frstlv06m']==0, 10, df_matriz_2['rec_frstlv06m'])

df_matriz_2['monto_final'] = df_matriz_2.monto_ec
df_matriz_2['tasa_final'] = df_matriz_2.tea_ec

df_matriz_2.shape

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

(135057, 260)

In [36]:
model = load_model('model_ela_pref_ec_1.h5')

with open("modas_ela_pref_ec_1.pkl" , 'rb') as file:  
    modas = pickle.load(file)
    
with open("scale_ela_pref_ec_1.pkl" , 'rb') as file:  
    scale = pickle.load(file)
    
columnas_cat = pd.read_pickle('col_cat_ela_pref_ec_1.pkl')   
columnas_num = pd.read_pickle('col_num_ela_pref_ec_1.pkl')   
columnas_tot = pd.read_pickle('col_ela_pref_ec_1.pkl') 

In [37]:
dfv = df_matriz_2.iloc[:,2:(len(df_matriz_2.columns))]

dfv.drop(['codmes2','tea_ec','monto_ec','segmento','orden'], inplace=True, axis=1)

dfv.tasa_final = dfv.tasa_final.fillna(dfv.tasa_final.mean())

percentiles = dfv.tasa_final.quantile([0.1,0.9]).values
dfv["tasa_final"][dfv["tasa_final"] <= percentiles[0]] = percentiles[0]
dfv["tasa_final"][dfv["tasa_final"] >= percentiles[1]] = percentiles[1]

flg_dfv = dfv[dfv.filter(regex='flg|flag').columns]
flg_dfv = flg_dfv.fillna(0)
flg_dfv = flg_dfv.apply(lambda x: x.astype('category'))
flg_dfv = pd.get_dummies(flg_dfv, drop_first=True)

dfvc = dfv[dfv.columns.difference(dfv.filter(regex='flg|flag|cod|mes|castiga').columns)]

dfvc = pd.concat([dfvc.select_dtypes(exclude=['int64','float64']),dfv[dfv.filter(regex='cod|mes|castiga').columns]],axis=1)
dfvc.replace('null',np.nan,inplace=True)
dfvc = dfvc[dfvc.columns.intersection(modas.index)]
for var in dfvc.columns:
    dfvc[var] = dfvc[var].fillna(modas[var])

dfvc = dfvc.apply(lambda x: x.astype('category'))

dfvc = pd.concat([pd.get_dummies(dfvc, drop_first=True),flg_dfv],axis=1)
dfvc_extra = pd.DataFrame(index=dfvc.index.copy(), columns=columnas_cat.columns.difference(dfvc.columns).intersection(columnas_tot.columns))
dfvc_extra = dfvc_extra.fillna(0) 

dfvc = pd.concat([dfvc,dfvc_extra],axis=1)

dfvn = dfv.select_dtypes(include=['int64','float64'])
dfvn = dfvn[dfvn.columns.difference(dfvn.filter(regex='flg|flag|cod|mes|castiga').columns)]  

dfvn['ing_bruto'].fillna((dfvn['ing_bruto'].mean()), inplace=True)

dfvn = dfvn.fillna(0)

dfvn = dfvn.mask((dfvn < dfvn.quantile(0.05)), dfvn.quantile(0.05), axis=1)  
dfvn = dfvn.mask((dfvn > dfvn.quantile(0.95)), dfvn.quantile(0.95), axis=1)  

dfvn_extra = pd.DataFrame(index=dfvn.index.copy(), columns=columnas_num.columns.difference(dfvn.columns))
dfvn_extra = dfvn_extra.fillna(0) 

dfvn = pd.concat([dfvn,dfvn_extra],axis=1)
dfvn = dfvn[dfvn.columns.intersection(columnas_num.columns)]
dfvn= dfvn[columnas_num.columns]

scaled_features_val = scale.transform(dfvn.values)
scaled_features_val_df = pd.DataFrame(scaled_features_val, index=dfvn.index, columns=dfvn.columns)

dfv1 = pd.concat([dfvc,scaled_features_val_df], axis=1)

dfv1 = dfv1[dfv1.columns.intersection(columnas_tot.columns)]

dfv1= dfv1[columnas_tot.columns]

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [114]:
std_mean_tasa = pd.read_csv('std_mean_tasa.csv')
std_mean_tasa.drop(std_mean_tasa.columns[0],axis=1,inplace=True)

In [115]:
std_mean_tasa

,tipo,std,mean
0,pref_1,10.946016,36.980660
1,pref_2,7.571852,24.027275
2,empr_1,9.660781,43.168562
3,empr_2,9.423123,31.932700


In [116]:
std_mean_tasa[std_mean_tasa.tipo=='pref_1']['mean']

0    36.98066
Name: mean, dtype: float64

In [38]:
model.load_weights("Best_ela_pref_ec_1.h5")

test = dfv1.copy()
x_temp = test.copy()
r = 0.5
x_temp.tasa_final = round(x_temp.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='pref_1']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='pref_1']['mean']),2)
tasa_prom=np.mean(x_temp.tasa_final)
proba_tasa = pd.DataFrame(index=test.index.copy())
tasas=[]

for i in range(1,80):
    
    x_temp.tasa_final = tasa_prom*r
    x_temp.tasa_final = (x_temp.tasa_final-float(std_mean_tasa[std_mean_tasa.tipo=='pref_1']['mean']))/float(std_mean_tasa[std_mean_tasa.tipo=='pref_1']['std'])
    prob = pd.DataFrame(model.predict_proba(x_temp),index=x_temp.index.copy(),columns=["prob"])
    proba_tasa = pd.concat([proba_tasa, pd.DataFrame(prob)], axis=1)
    tasas.append(tasa_prom*r)
    r = r+0.01

In [149]:
proba_tasa_back = proba_tasa.copy()

In [150]:
proba_tasa_back.head()

,prob,prob,prob,prob,prob,prob,prob,prob,prob,prob,...,prob,prob,prob,prob,prob,prob,prob,prob,prob,prob
2,0.691918,0.688246,0.684448,0.680694,0.676770,0.672756,0.668717,0.664667,0.660605,0.656500,...,0.328909,0.326881,0.324855,0.322854,0.320907,0.319000,0.317171,0.315400,0.313614,0.311822
9,0.638986,0.635943,0.632947,0.629969,0.626976,0.623975,0.621083,0.618233,0.615380,0.612557,...,0.365204,0.363288,0.361502,0.359977,0.358494,0.356990,0.355550,0.354146,0.352746,0.351256
15,0.611735,0.610786,0.609831,0.608844,0.607857,0.606869,0.605857,0.604795,0.603716,0.602605,...,0.509439,0.507236,0.504781,0.502300,0.499867,0.497253,0.494452,0.491654,0.488716,0.484995
17,0.793769,0.791846,0.789917,0.787972,0.786007,0.784031,0.782073,0.780079,0.777910,0.775723,...,0.571956,0.567311,0.562666,0.557879,0.553256,0.548622,0.543882,0.539239,0.534496,0.529499
19,0.931179,0.930227,0.929263,0.928296,0.927320,0.926330,0.925328,0.924310,0.923284,0.922246,...,0.816744,0.814126,0.811542,0.808932,0.806283,0.803614,0.800919,0.798118,0.795282,0.792414


In [39]:
proba_tasa.columns = tasas
proba_tasa.index=test.index.copy()

test = dfv1.copy()
test.tasa_final = round(test.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='pref_1']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='pref_1']['mean']),2)
proba_tasa = pd.concat([proba_tasa,test.tasa_final], axis=1)


In [40]:
tasa_iter = []
proba_iter = []
elasticidad = pd.DataFrame()
for i in range(0,(len(proba_tasa.columns)-10)):
    proba_iter=(proba_tasa.iloc[:,i+1]-proba_tasa.iloc[:,i])/proba_tasa.iloc[:,i]
    tasa_iter=(tasas[i+1]-tasas[i])/tasas[i]
    elasticidad=pd.concat([elasticidad, pd.DataFrame(proba_iter/tasa_iter)], axis=1)

elasticidad['elasticidad'] = elasticidad.mean(axis=1)    
   
proba_tasa= pd.concat([proba_tasa, elasticidad.elasticidad], axis=1)
proba_tasa.elasticidad = proba_tasa.elasticidad*-1


In [41]:
test = dfv1.copy()
x_temp = test.copy()
x_temp.index=test.index.copy()
prob = pd.DataFrame(model.predict_proba(x_temp),index=test.index.copy(),columns=["prob"])


In [42]:
proba_tasa = pd.concat([proba_tasa, prob,dfv.monto_final,df_matriz_2.codmes,df_matriz_2.key_value], axis=1)


In [43]:
proba_tasa.elasticidad.describe()

count    135057.000000
mean          0.452863
std           0.257594
min           0.001290
25%           0.266988
50%           0.465898
75%           0.590642
max           1.518940
Name: elasticidad, dtype: float64

In [44]:
proba_tasa.prob.describe()

count    135057.000000
mean          0.428775
std           0.286068
min           0.026179
25%           0.188883
50%           0.309531
75%           0.705384
max           0.997744
Name: prob, dtype: float64

In [45]:
proba_tasa_tmp_0 = proba_tasa.copy()
proba_tasa_tmp=proba_tasa_tmp_0.iloc[:,0:(len(proba_tasa_tmp_0.columns))-7].copy()

for i in range(0,(len(proba_tasa_tmp.columns))):
    proba_tasa_tmp.iloc[:,i]=round(abs(0.4-proba_tasa_tmp.iloc[:,i]),5)

minimos = proba_tasa_tmp.min(axis=1)
prob_tu=[]
n=0
for i in range(0,(proba_tasa_tmp.shape[0])):    
    n=0
    for j in range(0,(len(proba_tasa_tmp.columns))):
        if round(proba_tasa_tmp.iloc[i,j],5)==round(minimos.iloc[i],5) and n==0:
            prob_tu.append(proba_tasa_tmp.iloc[:,j].name)
            n=1

prob_tu=pd.DataFrame(prob_tu,index=proba_tasa_tmp.index.copy(),columns=["tu"])   

In [46]:
tempo = pd.concat([proba_tasa[proba_tasa.columns[0]],proba_tasa[proba_tasa.columns[len(proba_tasa.columns)-7]]],axis=1)

tempo.columns = ['before','after']

tempo['flg_elast'] = np.where(tempo.before>tempo.after,1,0)

tempo.flg_elast.value_counts()

1    135057
Name: flg_elast, dtype: int64

In [47]:
grupo_el = pd.concat([proba_tasa.codmes,proba_tasa.key_value,proba_tasa.prob,proba_tasa.elasticidad,prob_tu],axis=1)


In [49]:
grupo_el.prob.describe()

count    135057.000000
mean          0.428775
std           0.286068
min           0.026179
25%           0.188883
50%           0.309531
75%           0.705384
max           0.997744
Name: prob, dtype: float64

In [50]:
grupo_el.head()

,codmes,key_value,prob,elasticidad,tu
2,201907,C5C79D500A603156826227F156F77CB25A7433C4B37DCB...,0.529112,0.922916,38.300180
9,201907,5D227E2279E1C8CDBF0877AFFE65E2686CFA10E5633540...,0.336597,0.727085,42.734937
15,201909,A083EB7D8CE62EF5A15DA7E543AEED52BBB78601D6BD11...,0.594864,0.245199,51.201293
17,201907,087048832FE869267E5D0621D2F692DC8E59B7067AB436...,0.404346,0.439773,51.201293
19,201907,047FA52404C608BCB29557D93E21FBE3381179CDC23E70...,0.849888,0.175311,51.201293


In [51]:
grupo_el.to_csv('grupo_el_pref_1_back.csv',index=False)

In [5]:
grupo_el_pref1 = pd.read_csv('grupo_el_pref_1_back.csv')

In [7]:
grupo_el_pref1.shape

(135057, 5)

In [9]:
grupo_el_pref1.drop(grupo_el_pref1.columns[0],axis=1,inplace=True)

In [10]:
grupo_el_pref1.head()

,key_value,prob,elasticidad,tu
0,C5C79D500A603156826227F156F77CB25A7433C4B37DCB...,0.529112,0.922916,38.300180
1,5D227E2279E1C8CDBF0877AFFE65E2686CFA10E5633540...,0.336597,0.727085,42.734937
2,A083EB7D8CE62EF5A15DA7E543AEED52BBB78601D6BD11...,0.594864,0.245199,51.201293
3,087048832FE869267E5D0621D2F692DC8E59B7067AB436...,0.404346,0.439773,51.201293
4,047FA52404C608BCB29557D93E21FBE3381179CDC23E70...,0.849888,0.175311,51.201293


In [12]:
df_matriz_2 = df[(df.segmento==2) & (df.monto_ec>16000)]

df_matriz_2['mes_ultprd'] = df_matriz_2['mes_ultprd'].fillna(df_matriz_2.codmes)
df_matriz_2['rec_ultprd'] = ((pd.to_datetime(df_matriz_2['codmes'], format='%Y%m') - pd.to_datetime(df_matriz_2['mes_ultprd'].astype('int').astype('str'), format='%Y%m')) / np.timedelta64(1, 'D')).astype('int64') 

df_matriz_2['sow_ibk'] = df_matriz_2.salvig_tot_rccibk_m01/df_matriz_2.salvig_tot_rccsf_m01
df_matriz_2['cem_2'] = df_matriz_2.salvig_tot_rccsf_m01/df_matriz_2.ing_bruto

df_matriz_2.rec_frstlv06m = df_matriz_2.rec_frstlv06m.fillna(0)
df_matriz_2['rec_frstlv06m'] = np.where(df_matriz_2['rec_frstlv06m']==0, 10, df_matriz_2['rec_frstlv06m'])

df_matriz_2['monto_final'] = df_matriz_2.monto_ec
df_matriz_2['tasa_final'] = df_matriz_2.tea_ec

df_matriz_2.shape

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

(110268, 260)

In [13]:
model = load_model('model_ela_pref_ec_2.h5')

with open("modas_ela_pref_ec_2.pkl" , 'rb') as file:  
    modas = pickle.load(file)
    
with open("scale_ela_pref_ec_2.pkl" , 'rb') as file:  
    scale = pickle.load(file)
    
columnas_cat = pd.read_pickle('col_cat_ela_pref_ec_2.pkl')   
columnas_num = pd.read_pickle('col_num_ela_pref_ec_2.pkl')   
columnas_tot = pd.read_pickle('col_ela_pref_ec_2.pkl') 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [14]:
dfv = df_matriz_2.iloc[:,2:(len(df_matriz_2.columns))]

dfv.drop(['codmes2','tea_ec','monto_ec','segmento','orden'], inplace=True, axis=1)

dfv.tasa_final = dfv.tasa_final.fillna(dfv.tasa_final.mean())

percentiles = dfv.tasa_final.quantile([0.1,0.9]).values
dfv["tasa_final"][dfv["tasa_final"] <= percentiles[0]] = percentiles[0]
dfv["tasa_final"][dfv["tasa_final"] >= percentiles[1]] = percentiles[1]

flg_dfv = dfv[dfv.filter(regex='flg|flag').columns]
flg_dfv = flg_dfv.fillna(0)
flg_dfv = flg_dfv.apply(lambda x: x.astype('category'))
flg_dfv = pd.get_dummies(flg_dfv, drop_first=True)

dfvc = dfv[dfv.columns.difference(dfv.filter(regex='flg|flag|cod|mes|castiga').columns)]

dfvc = pd.concat([dfvc.select_dtypes(exclude=['int64','float64']),dfv[dfv.filter(regex='cod|mes|castiga').columns]],axis=1)
dfvc.replace('null',np.nan,inplace=True)
dfvc = dfvc[dfvc.columns.intersection(modas.index)]
for var in dfvc.columns:
    dfvc[var] = dfvc[var].fillna(modas[var])

dfvc = dfvc.apply(lambda x: x.astype('category'))

dfvc = pd.concat([pd.get_dummies(dfvc, drop_first=True),flg_dfv],axis=1)
dfvc_extra = pd.DataFrame(index=dfvc.index.copy(), columns=columnas_cat.columns.difference(dfvc.columns).intersection(columnas_tot.columns))
dfvc_extra = dfvc_extra.fillna(0) 

dfvc = pd.concat([dfvc,dfvc_extra],axis=1)

dfvn = dfv.select_dtypes(include=['int64','float64'])
dfvn = dfvn[dfvn.columns.difference(dfvn.filter(regex='flg|flag|cod|mes|castiga').columns)]  

dfvn['ing_bruto'].fillna((dfvn['ing_bruto'].mean()), inplace=True)

dfvn = dfvn.fillna(0)

dfvn = dfvn.mask((dfvn < dfvn.quantile(0.05)), dfvn.quantile(0.05), axis=1)  
dfvn = dfvn.mask((dfvn > dfvn.quantile(0.95)), dfvn.quantile(0.95), axis=1)  

dfvn_extra = pd.DataFrame(index=dfvn.index.copy(), columns=columnas_num.columns.difference(dfvn.columns))
dfvn_extra = dfvn_extra.fillna(0) 

dfvn = pd.concat([dfvn,dfvn_extra],axis=1)
dfvn = dfvn[dfvn.columns.intersection(columnas_num.columns)]
dfvn= dfvn[columnas_num.columns]

scaled_features_val = scale.transform(dfvn.values)
scaled_features_val_df = pd.DataFrame(scaled_features_val, index=dfvn.index, columns=dfvn.columns)

dfv1 = pd.concat([dfvc,scaled_features_val_df], axis=1)

dfv1 = dfv1[dfv1.columns.intersection(columnas_tot.columns)]

dfv1= dfv1[columnas_tot.columns]

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
dfv1.shape

(110268, 199)

In [16]:
std_mean_tasa = pd.read_csv('std_mean_tasa.csv')
std_mean_tasa.drop(std_mean_tasa.columns[0],axis=1,inplace=True)
std_mean_tasa

,tipo,std,mean
0,pref_1,10.946016,36.980660
1,pref_2,7.571852,24.027275
2,empr_1,9.660781,43.168562
3,empr_2,9.423123,31.932700


In [17]:
model.load_weights("Best_ela_pref_ec_2.h5")

test = dfv1.copy()
x_temp = test.copy()
r = 0.5
x_temp.tasa_final = round(x_temp.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='pref_2']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='pref_2']['mean']),2)
tasa_prom=np.mean(x_temp.tasa_final)
proba_tasa = pd.DataFrame(index=test.index.copy())
tasas=[]

for i in range(1,80):
    
    x_temp.tasa_final = tasa_prom*r
    x_temp.tasa_final = (x_temp.tasa_final-float(std_mean_tasa[std_mean_tasa.tipo=='pref_2']['mean']))/float(std_mean_tasa[std_mean_tasa.tipo=='pref_2']['std'])
    prob = pd.DataFrame(model.predict_proba(x_temp),index=x_temp.index.copy(),columns=["prob"])
    proba_tasa = pd.concat([proba_tasa, pd.DataFrame(prob)], axis=1)
    tasas.append(tasa_prom*r)
    r = r+0.01

In [18]:
proba_tasa.columns = tasas
proba_tasa.index=test.index.copy()

test = dfv1.copy()
test.tasa_final = round(test.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='pref_2']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='pref_2']['mean']),2)
proba_tasa = pd.concat([proba_tasa,test.tasa_final], axis=1)

tasa_iter = []
proba_iter = []
elasticidad = pd.DataFrame()
for i in range(0,(len(proba_tasa.columns)-10)):
    proba_iter=(proba_tasa.iloc[:,i+1]-proba_tasa.iloc[:,i])/proba_tasa.iloc[:,i]
    tasa_iter=(tasas[i+1]-tasas[i])/tasas[i]
    elasticidad=pd.concat([elasticidad, pd.DataFrame(proba_iter/tasa_iter)], axis=1)

elasticidad['elasticidad'] = elasticidad.mean(axis=1)    
   
proba_tasa= pd.concat([proba_tasa, elasticidad.elasticidad], axis=1)
proba_tasa.elasticidad = proba_tasa.elasticidad*-1

test = dfv1.copy()
x_temp = test.copy()
x_temp.index=test.index.copy()
prob = pd.DataFrame(model.predict_proba(x_temp),index=test.index.copy(),columns=["prob"])

proba_tasa = pd.concat([proba_tasa, prob,dfv.monto_final,df_matriz_2.codmes,df_matriz_2.key_value], axis=1)

proba_tasa_tmp_0 = proba_tasa.copy()
proba_tasa_tmp=proba_tasa_tmp_0.iloc[:,0:(len(proba_tasa_tmp_0.columns))-7].copy()

for i in range(0,(len(proba_tasa_tmp.columns))):
    proba_tasa_tmp.iloc[:,i]=round(abs(0.4-proba_tasa_tmp.iloc[:,i]),5)

minimos = proba_tasa_tmp.min(axis=1)
prob_tu=[]
n=0
for i in range(0,(proba_tasa_tmp.shape[0])):    
    n=0
    for j in range(0,(len(proba_tasa_tmp.columns))):
        if round(proba_tasa_tmp.iloc[i,j],5)==round(minimos.iloc[i],5) and n==0:
            prob_tu.append(proba_tasa_tmp.iloc[:,j].name)
            n=1

prob_tu=pd.DataFrame(prob_tu,index=proba_tasa_tmp.index.copy(),columns=["tu"])  

tempo = pd.concat([proba_tasa[proba_tasa.columns[0]],proba_tasa[proba_tasa.columns[len(proba_tasa.columns)-7]]],axis=1)
tempo.columns = ['before','after']
tempo['flg_elast'] = np.where(tempo.before>tempo.after,1,0)

tempo.flg_elast.value_counts()

grupo_el = pd.concat([proba_tasa.codmes,proba_tasa.key_value,proba_tasa.prob,proba_tasa.elasticidad,prob_tu],axis=1)

grupo_el.to_csv('grupo_el_pref_2_back.csv',index=False)

In [19]:
grupo_el.head()

,codmes,key_value,prob,elasticidad,tu
1,201909,7B290C54B86126665F8630762305BFF787EEB75CB71FE3...,0.448742,0.541166,21.474917
6,201907,47BC4B38C1C817B6B47268171402E0159218104FCD47CC...,0.534690,0.398573,32.468030
7,201909,2004BEBBBFC968471C3B42A6C027AF6A9E220EC203D907...,0.301837,0.537374,12.782689
8,201907,F287DC1F81D77B8152C771460450B5D27A6234C7F80B1B...,0.714473,0.090965,32.468030
13,201907,E25C619CCF0FFEDE3EA36617A7AB96E998E8EBF8F813EB...,0.714330,0.005883,32.468030


In [20]:
grupo_el.prob.describe()

count    110268.000000
mean          0.458945
std           0.140175
min           0.097254
25%           0.353675
50%           0.439821
75%           0.545689
max           0.714823
Name: prob, dtype: float64

In [21]:
grupo_el.elasticidad.describe()

count    110268.000000
mean          0.457520
std           0.184477
min          -0.000486
25%           0.388801
50%           0.474188
75%           0.568827
max           1.046689
Name: elasticidad, dtype: float64

In [22]:
df_matriz_2 = df[(df.segmento==3) & (df.monto_ec<=14000)]

df_matriz_2['mes_ultprd'] = df_matriz_2['mes_ultprd'].fillna(df_matriz_2.codmes)
df_matriz_2['rec_ultprd'] = ((pd.to_datetime(df_matriz_2['codmes'], format='%Y%m') - pd.to_datetime(df_matriz_2['mes_ultprd'].astype('int').astype('str'), format='%Y%m')) / np.timedelta64(1, 'D')).astype('int64') 

df_matriz_2['sow_ibk'] = df_matriz_2.salvig_tot_rccibk_m01/df_matriz_2.salvig_tot_rccsf_m01
df_matriz_2['cem_2'] = df_matriz_2.salvig_tot_rccsf_m01/df_matriz_2.ing_bruto

df_matriz_2.rec_frstlv06m = df_matriz_2.rec_frstlv06m.fillna(0)
df_matriz_2['rec_frstlv06m'] = np.where(df_matriz_2['rec_frstlv06m']==0, 10, df_matriz_2['rec_frstlv06m'])

df_matriz_2['monto_final'] = df_matriz_2.monto_ec
df_matriz_2['tasa_final'] = df_matriz_2.tea_ec

model = load_model('model_ela_empr_ec_1.h5')

with open("modas_ela_empr_ec_1.pkl" , 'rb') as file:  
    modas = pickle.load(file)
    
with open("scale_ela_empr_ec_1.pkl" , 'rb') as file:  
    scale = pickle.load(file)
    
columnas_cat = pd.read_pickle('col_cat_ela_empr_ec_1.pkl')   
columnas_num = pd.read_pickle('col_num_ela_empr_ec_1.pkl')   
columnas_tot = pd.read_pickle('col_ela_empr_ec_1.pkl') 


dfv = df_matriz_2.iloc[:,2:(len(df_matriz_2.columns))]

dfv.drop(['codmes2','tea_ec','monto_ec','segmento','orden'], inplace=True, axis=1)

dfv.tasa_final = dfv.tasa_final.fillna(dfv.tasa_final.mean())

percentiles = dfv.tasa_final.quantile([0.1,0.9]).values
dfv["tasa_final"][dfv["tasa_final"] <= percentiles[0]] = percentiles[0]
dfv["tasa_final"][dfv["tasa_final"] >= percentiles[1]] = percentiles[1]

flg_dfv = dfv[dfv.filter(regex='flg|flag').columns]
flg_dfv = flg_dfv.fillna(0)
flg_dfv = flg_dfv.apply(lambda x: x.astype('category'))
flg_dfv = pd.get_dummies(flg_dfv, drop_first=True)

dfvc = dfv[dfv.columns.difference(dfv.filter(regex='flg|flag|cod|mes|castiga').columns)]

dfvc = pd.concat([dfvc.select_dtypes(exclude=['int64','float64']),dfv[dfv.filter(regex='cod|mes|castiga').columns]],axis=1)
dfvc.replace('null',np.nan,inplace=True)
dfvc = dfvc[dfvc.columns.intersection(modas.index)]
for var in dfvc.columns:
    dfvc[var] = dfvc[var].fillna(modas[var])

dfvc = dfvc.apply(lambda x: x.astype('category'))

dfvc = pd.concat([pd.get_dummies(dfvc, drop_first=True),flg_dfv],axis=1)
dfvc_extra = pd.DataFrame(index=dfvc.index.copy(), columns=columnas_cat.columns.difference(dfvc.columns).intersection(columnas_tot.columns))
dfvc_extra = dfvc_extra.fillna(0) 

dfvc = pd.concat([dfvc,dfvc_extra],axis=1)

dfvn = dfv.select_dtypes(include=['int64','float64'])
dfvn = dfvn[dfvn.columns.difference(dfvn.filter(regex='flg|flag|cod|mes|castiga').columns)]  

dfvn['ing_bruto'].fillna((dfvn['ing_bruto'].mean()), inplace=True)

dfvn = dfvn.fillna(0)

dfvn = dfvn.mask((dfvn < dfvn.quantile(0.05)), dfvn.quantile(0.05), axis=1)  
dfvn = dfvn.mask((dfvn > dfvn.quantile(0.95)), dfvn.quantile(0.95), axis=1)  

dfvn_extra = pd.DataFrame(index=dfvn.index.copy(), columns=columnas_num.columns.difference(dfvn.columns))
dfvn_extra = dfvn_extra.fillna(0) 

dfvn = pd.concat([dfvn,dfvn_extra],axis=1)
dfvn = dfvn[dfvn.columns.intersection(columnas_num.columns)]
dfvn= dfvn[columnas_num.columns]

scaled_features_val = scale.transform(dfvn.values)
scaled_features_val_df = pd.DataFrame(scaled_features_val, index=dfvn.index, columns=dfvn.columns)

dfv1 = pd.concat([dfvc,scaled_features_val_df], axis=1)

dfv1 = dfv1[dfv1.columns.intersection(columnas_tot.columns)]

dfv1= dfv1[columnas_tot.columns]


model.load_weights("Best_ela_empr_ec_1.h5")

test = dfv1.copy()
x_temp = test.copy()
r = 0.5
x_temp.tasa_final = round(x_temp.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='empr_1']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='empr_1']['mean']),2)
tasa_prom=np.mean(x_temp.tasa_final)
proba_tasa = pd.DataFrame(index=test.index.copy())
tasas=[]

for i in range(1,80):
    
    x_temp.tasa_final = tasa_prom*r
    x_temp.tasa_final = (x_temp.tasa_final-float(std_mean_tasa[std_mean_tasa.tipo=='empr_1']['mean']))/float(std_mean_tasa[std_mean_tasa.tipo=='empr_1']['std'])
    prob = pd.DataFrame(model.predict_proba(x_temp),index=x_temp.index.copy(),columns=["prob"])
    proba_tasa = pd.concat([proba_tasa, pd.DataFrame(prob)], axis=1)
    tasas.append(tasa_prom*r)
    r = r+0.01


proba_tasa.columns = tasas
proba_tasa.index=test.index.copy()

test = dfv1.copy()
test.tasa_final = round(test.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='empr_1']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='empr_1']['mean']),2)
proba_tasa = pd.concat([proba_tasa,test.tasa_final], axis=1)

tasa_iter = []
proba_iter = []
elasticidad = pd.DataFrame()
for i in range(0,(len(proba_tasa.columns)-10)):
    proba_iter=(proba_tasa.iloc[:,i+1]-proba_tasa.iloc[:,i])/proba_tasa.iloc[:,i]
    tasa_iter=(tasas[i+1]-tasas[i])/tasas[i]
    elasticidad=pd.concat([elasticidad, pd.DataFrame(proba_iter/tasa_iter)], axis=1)

elasticidad['elasticidad'] = elasticidad.mean(axis=1)    
   
proba_tasa= pd.concat([proba_tasa, elasticidad.elasticidad], axis=1)
proba_tasa.elasticidad = proba_tasa.elasticidad*-1

test = dfv1.copy()
x_temp = test.copy()
x_temp.index=test.index.copy()
prob = pd.DataFrame(model.predict_proba(x_temp),index=test.index.copy(),columns=["prob"])

proba_tasa = pd.concat([proba_tasa, prob,dfv.monto_final,df_matriz_2.codmes,df_matriz_2.key_value], axis=1)

proba_tasa_tmp_0 = proba_tasa.copy()
proba_tasa_tmp=proba_tasa_tmp_0.iloc[:,0:(len(proba_tasa_tmp_0.columns))-7].copy()

for i in range(0,(len(proba_tasa_tmp.columns))):
    proba_tasa_tmp.iloc[:,i]=round(abs(0.4-proba_tasa_tmp.iloc[:,i]),5)

minimos = proba_tasa_tmp.min(axis=1)
prob_tu=[]
n=0
for i in range(0,(proba_tasa_tmp.shape[0])):    
    n=0
    for j in range(0,(len(proba_tasa_tmp.columns))):
        if round(proba_tasa_tmp.iloc[i,j],5)==round(minimos.iloc[i],5) and n==0:
            prob_tu.append(proba_tasa_tmp.iloc[:,j].name)
            n=1

prob_tu=pd.DataFrame(prob_tu,index=proba_tasa_tmp.index.copy(),columns=["tu"])  

tempo = pd.concat([proba_tasa[proba_tasa.columns[0]],proba_tasa[proba_tasa.columns[len(proba_tasa.columns)-7]]],axis=1)
tempo.columns = ['before','after']
tempo['flg_elast'] = np.where(tempo.before>tempo.after,1,0)

tempo.flg_elast.value_counts()

grupo_el = pd.concat([proba_tasa.codmes,proba_tasa.key_value,proba_tasa.prob,proba_tasa.elasticidad,prob_tu],axis=1)

grupo_el.to_csv('grupo_el_empr_1_back.csv',index=False)

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [23]:
df_matriz_2 = df[(df.segmento==3) & (df.monto_ec>14000)]

df_matriz_2['mes_ultprd'] = df_matriz_2['mes_ultprd'].fillna(df_matriz_2.codmes)
df_matriz_2['rec_ultprd'] = ((pd.to_datetime(df_matriz_2['codmes'], format='%Y%m') - pd.to_datetime(df_matriz_2['mes_ultprd'].astype('int').astype('str'), format='%Y%m')) / np.timedelta64(1, 'D')).astype('int64') 

df_matriz_2['sow_ibk'] = df_matriz_2.salvig_tot_rccibk_m01/df_matriz_2.salvig_tot_rccsf_m01
df_matriz_2['cem_2'] = df_matriz_2.salvig_tot_rccsf_m01/df_matriz_2.ing_bruto

df_matriz_2.rec_frstlv06m = df_matriz_2.rec_frstlv06m.fillna(0)
df_matriz_2['rec_frstlv06m'] = np.where(df_matriz_2['rec_frstlv06m']==0, 10, df_matriz_2['rec_frstlv06m'])

df_matriz_2['monto_final'] = df_matriz_2.monto_ec
df_matriz_2['tasa_final'] = df_matriz_2.tea_ec

model = load_model('model_ela_empr_ec_2.h5')

with open("modas_ela_empr_ec_2.pkl" , 'rb') as file:  
    modas = pickle.load(file)
    
with open("scale_ela_empr_ec_2.pkl" , 'rb') as file:  
    scale = pickle.load(file)
    
columnas_cat = pd.read_pickle('col_cat_ela_empr_ec_2.pkl')   
columnas_num = pd.read_pickle('col_num_ela_empr_ec_2.pkl')   
columnas_tot = pd.read_pickle('col_ela_empr_ec_2.pkl') 


dfv = df_matriz_2.iloc[:,2:(len(df_matriz_2.columns))]

dfv.drop(['codmes2','tea_ec','monto_ec','segmento','orden'], inplace=True, axis=1)

dfv.tasa_final = dfv.tasa_final.fillna(dfv.tasa_final.mean())

percentiles = dfv.tasa_final.quantile([0.1,0.9]).values
dfv["tasa_final"][dfv["tasa_final"] <= percentiles[0]] = percentiles[0]
dfv["tasa_final"][dfv["tasa_final"] >= percentiles[1]] = percentiles[1]

flg_dfv = dfv[dfv.filter(regex='flg|flag').columns]
flg_dfv = flg_dfv.fillna(0)
flg_dfv = flg_dfv.apply(lambda x: x.astype('category'))
flg_dfv = pd.get_dummies(flg_dfv, drop_first=True)

dfvc = dfv[dfv.columns.difference(dfv.filter(regex='flg|flag|cod|mes|castiga').columns)]

dfvc = pd.concat([dfvc.select_dtypes(exclude=['int64','float64']),dfv[dfv.filter(regex='cod|mes|castiga').columns]],axis=1)
dfvc.replace('null',np.nan,inplace=True)
dfvc = dfvc[dfvc.columns.intersection(modas.index)]
for var in dfvc.columns:
    dfvc[var] = dfvc[var].fillna(modas[var])

dfvc = dfvc.apply(lambda x: x.astype('category'))

dfvc = pd.concat([pd.get_dummies(dfvc, drop_first=True),flg_dfv],axis=1)
dfvc_extra = pd.DataFrame(index=dfvc.index.copy(), columns=columnas_cat.columns.difference(dfvc.columns).intersection(columnas_tot.columns))
dfvc_extra = dfvc_extra.fillna(0) 

dfvc = pd.concat([dfvc,dfvc_extra],axis=1)

dfvn = dfv.select_dtypes(include=['int64','float64'])
dfvn = dfvn[dfvn.columns.difference(dfvn.filter(regex='flg|flag|cod|mes|castiga').columns)]  

dfvn['ing_bruto'].fillna((dfvn['ing_bruto'].mean()), inplace=True)

dfvn = dfvn.fillna(0)

dfvn = dfvn.mask((dfvn < dfvn.quantile(0.05)), dfvn.quantile(0.05), axis=1)  
dfvn = dfvn.mask((dfvn > dfvn.quantile(0.95)), dfvn.quantile(0.95), axis=1)  

dfvn_extra = pd.DataFrame(index=dfvn.index.copy(), columns=columnas_num.columns.difference(dfvn.columns))
dfvn_extra = dfvn_extra.fillna(0) 

dfvn = pd.concat([dfvn,dfvn_extra],axis=1)
dfvn = dfvn[dfvn.columns.intersection(columnas_num.columns)]
dfvn= dfvn[columnas_num.columns]

scaled_features_val = scale.transform(dfvn.values)
scaled_features_val_df = pd.DataFrame(scaled_features_val, index=dfvn.index, columns=dfvn.columns)

dfv1 = pd.concat([dfvc,scaled_features_val_df], axis=1)

dfv1 = dfv1[dfv1.columns.intersection(columnas_tot.columns)]

dfv1= dfv1[columnas_tot.columns]


model.load_weights("Best_ela_empr_ec_2.h5")

test = dfv1.copy()
x_temp = test.copy()
r = 0.5
x_temp.tasa_final = round(x_temp.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='empr_2']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='empr_2']['mean']),2)
tasa_prom=np.mean(x_temp.tasa_final)
proba_tasa = pd.DataFrame(index=test.index.copy())
tasas=[]

for i in range(1,80):
    
    x_temp.tasa_final = tasa_prom*r
    x_temp.tasa_final = (x_temp.tasa_final-float(std_mean_tasa[std_mean_tasa.tipo=='empr_2']['mean']))/float(std_mean_tasa[std_mean_tasa.tipo=='empr_2']['std'])
    prob = pd.DataFrame(model.predict_proba(x_temp),index=x_temp.index.copy(),columns=["prob"])
    proba_tasa = pd.concat([proba_tasa, pd.DataFrame(prob)], axis=1)
    tasas.append(tasa_prom*r)
    r = r+0.01


proba_tasa.columns = tasas
proba_tasa.index=test.index.copy()

test = dfv1.copy()
test.tasa_final = round(test.tasa_final*float(std_mean_tasa[std_mean_tasa.tipo=='empr_2']['std'])+float(std_mean_tasa[std_mean_tasa.tipo=='empr_2']['mean']),2)
proba_tasa = pd.concat([proba_tasa,test.tasa_final], axis=1)

tasa_iter = []
proba_iter = []
elasticidad = pd.DataFrame()
for i in range(0,(len(proba_tasa.columns)-10)):
    proba_iter=(proba_tasa.iloc[:,i+1]-proba_tasa.iloc[:,i])/proba_tasa.iloc[:,i]
    tasa_iter=(tasas[i+1]-tasas[i])/tasas[i]
    elasticidad=pd.concat([elasticidad, pd.DataFrame(proba_iter/tasa_iter)], axis=1)

elasticidad['elasticidad'] = elasticidad.mean(axis=1)    
   
proba_tasa= pd.concat([proba_tasa, elasticidad.elasticidad], axis=1)
proba_tasa.elasticidad = proba_tasa.elasticidad*-1

test = dfv1.copy()
x_temp = test.copy()
x_temp.index=test.index.copy()
prob = pd.DataFrame(model.predict_proba(x_temp),index=test.index.copy(),columns=["prob"])

proba_tasa = pd.concat([proba_tasa, prob,dfv.monto_final,df_matriz_2.codmes,df_matriz_2.key_value], axis=1)

proba_tasa_tmp_0 = proba_tasa.copy()
proba_tasa_tmp=proba_tasa_tmp_0.iloc[:,0:(len(proba_tasa_tmp_0.columns))-7].copy()

for i in range(0,(len(proba_tasa_tmp.columns))):
    proba_tasa_tmp.iloc[:,i]=round(abs(0.4-proba_tasa_tmp.iloc[:,i]),5)

minimos = proba_tasa_tmp.min(axis=1)
prob_tu=[]
n=0
for i in range(0,(proba_tasa_tmp.shape[0])):    
    n=0
    for j in range(0,(len(proba_tasa_tmp.columns))):
        if round(proba_tasa_tmp.iloc[i,j],5)==round(minimos.iloc[i],5) and n==0:
            prob_tu.append(proba_tasa_tmp.iloc[:,j].name)
            n=1

prob_tu=pd.DataFrame(prob_tu,index=proba_tasa_tmp.index.copy(),columns=["tu"])  

tempo = pd.concat([proba_tasa[proba_tasa.columns[0]],proba_tasa[proba_tasa.columns[len(proba_tasa.columns)-7]]],axis=1)
tempo.columns = ['before','after']
tempo['flg_elast'] = np.where(tempo.before>tempo.after,1,0)

tempo.flg_elast.value_counts()

grupo_el = pd.concat([proba_tasa.codmes,proba_tasa.key_value,proba_tasa.prob,proba_tasa.elasticidad,prob_tu],axis=1)

grupo_el.to_csv('grupo_el_empr_2_back.csv',index=False)

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [24]:
import pandas as pd

grupo_el_1 = pd.read_csv('grupo_el_pref_1_back.csv')
grupo_el_2 = pd.read_csv('grupo_el_pref_2_back.csv')
grupo_el_3 = pd.read_csv('grupo_el_empr_1_back.csv')
grupo_el_4 = pd.read_csv('grupo_el_empr_2_back.csv')

grupo_el_total = grupo_el_1.append(grupo_el_2).append(grupo_el_3).append(grupo_el_4)


In [25]:
grupo_el_total.head()

,codmes,key_value,prob,elasticidad,tu
0,201907,C5C79D500A603156826227F156F77CB25A7433C4B37DCB...,0.529112,0.922916,38.300180
1,201907,5D227E2279E1C8CDBF0877AFFE65E2686CFA10E5633540...,0.336597,0.727085,42.734937
2,201909,A083EB7D8CE62EF5A15DA7E543AEED52BBB78601D6BD11...,0.594864,0.245199,51.201293
3,201907,087048832FE869267E5D0621D2F692DC8E59B7067AB436...,0.404346,0.439773,51.201293
4,201907,047FA52404C608BCB29557D93E21FBE3381179CDC23E70...,0.849888,0.175311,51.201293


In [26]:
grupo_el_total.codmes.value_counts()

201907    197608
201908    194762
201909    176095
Name: codmes, dtype: int64

In [27]:
grupo_el_4.head()

,codmes,key_value,prob,elasticidad,tu
0,201907,0908A7F013F857CE0F58F913DF5A5C78ABDA780DA3859F...,0.500827,0.430276,44.687563
1,201907,DFED6F5EC3CC5F78C767058F4EACDF3F0A7B7C7BF291AF...,0.428458,0.433461,44.687563
2,201907,B174953A0587C2ADC844F2E1B15227D98A65EA9EB301DC...,0.505053,0.525432,42.576340
3,201909,CD7B866E451672C0F21A0FCC16187BFB16ADAA07931DF6...,0.324738,0.503165,36.946411
4,201909,64FB30087FE4CAED293B191F39EBC8EF2BFAB47B7434C6...,0.430290,0.440628,39.057634


In [28]:
grupo_el_total.shape

(568465, 5)

In [29]:
grupo_el_total.to_csv('elast_ec_back.csv',index=False)